In [ ]:
from numpy.lib.stride_tricks import sliding_window_view

def create_dataset_vectorized(df, lookback_minutes=30):
    data = df.values
    dataX = sliding_window_view(data, window_shape=(lookback_minutes, data.shape[1]))
    dataY = data[lookback_minutes:]
    return dataX, dataY

# Assume df is your DataFrame where each row represents a minute
df = pd.read_csv("your_data_file.csv")  # Load your data here
X, Y = create_dataset_vectorized(df, lookback_minutes=30)


In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Define constants
FEATURES_PER_MINUTE = 25
LOOKBACK_MINUTES = 30
INPUT_SHAPE = (LOOKBACK_MINUTES, FEATURES_PER_MINUTE)

# Initialize the model
model = Sequential()

# Add LSTM layer with 50 units
# return_sequences=True makes LSTM layer return full sequence instead of last output only
model.add(LSTM(50, return_sequences=True, input_shape=INPUT_SHAPE))

# You can add more LSTM or Dense layers here if needed

# Add a Dense layer with single unit as final layer
# This assumes a regression task; for classification, you might need a different activation and number of units
model.add(Dense(1))

# Compile the model
model.compile(optimizer='rmsprop', loss='mse')  # adjust optimizer and loss as needed


In [ ]:
import tensorflow as tf

def create_lstm_model(input_shape, num_lstm_units=64, num_dense_units=32, dropout_rate=0.2):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(num_lstm_units, input_shape=input_shape))
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Dense(num_dense_units, activation='relu'))
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Dense(1, activation='linear'))  # Assuming you're doing regression

    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# Assuming your input shape is (sequence_length, num_features)
input_shape = (sequence_length, num_features)

model = create_lstm_model(input_shape)


In [ ]:
import tensorflow as tf

def create_time_series_transformer(input_shape, num_transformer_units=64, num_dense_units=32, num_heads=4, dropout_rate=0.2):
    input_layer = tf.keras.layers.Input(shape=input_shape)
    transformer_output = tf.keras.layers.MultiHeadAttention(key_dim=num_transformer_units, num_heads=num_heads)(input_layer, input_layer)
    transformer_output = tf.keras.layers.Dropout(dropout_rate)(transformer_output)
    transformer_output = tf.keras.layers.LayerNormalization(epsilon=1e-6)(transformer_output)
    transformer_output = tf.keras.layers.GlobalAveragePooling1D()(transformer_output)
    
    dense_layer = tf.keras.layers.Dense(num_dense_units, activation='relu')(transformer_output)
    dropout_layer = tf.keras.layers.Dropout(dropout_rate)(dense_layer)
    output_layer = tf.keras.layers.Dense(1, activation='linear')(dropout_layer)
    
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    return model

# Assuming your input shape is (sequence_length, num_features)
input_shape = (sequence_length, num_features)

model = create_time_series_transformer(input_shape)
